# Homework 1: Data Validation and Transformation via Polars DataFrames

## Background

You work in Supply Chain for a hyperscale cloud provider. One of your organization's responsibilities is to procure server racks for data centers. Server racks have two major categories of components: server components and rack components. Server components make up each individual server (e.g., CPU, SSD, HDD, etc.). There can be multiple servers in a rack. Rack components hold all the servers in a chassis and provide a top-of-rack (TOR) networking switch for data center connectivity.

## Situation

We manage several different server rack programs for various compute products. Each program is quoted by a variety of vendors. The quoting process is challenging. Initially, vendors submitted summary-level quotes that provided the cost of an entire server rack. As your organization evolved, you asked vendors to start providing more detailed quotes where each line represents the cost of a individual component in the server rack. These quote line details do not include server or rack quantities. Unfortunately, internal systems are still tied to the summary-level quote submissions. The current state requires vendors to provide quotes as both summaries and detailed line items.
Vendors have built automation to continuously supply quotes, as market rates for various components can fluctuate daily. 

## Available Data

- Server component quantites (Excel)
- Rack component quantities (Excel)
- Vendor quote summary data (csv)
- Vendor quote line detail data (csv)

# Import Libraries and Inspect Datasets

This section imports all four source datasets and removes the "Server" label on the program columns from the excel dataframes (server_specs_raw and rack_specs_raw).

All four datasets come from Data 516 - Scalable Algorithms taught by professor Mark Kazzaz.

In [3]:
%%capture
%pip install polars
%pip install pandas
%pip install fastexcel
%pip install pyarrow
%pip install openpyxl

import pyarrow
import fastexcel
import polars as pl
import pandas as pd
from datetime import datetime

# Assignment

As part of the data team, you need to transform various datasets to create the following:

- A table detailing the total server rack quantity for each component across all programs. 

    - Each record in the table should represent a distinct component. 
    - Each attribute of the table should represent a distinct program. 
    - The intersection of record and attribute should represent the total extended quantity of parts for that component and program combination.

- You must join the relevant datasets and apply the necessary calculations, transformations, and filters to create a final dataset that complies with all of our data validation requirements.

- Once you have a dataset that meets all the requirements, create tables for each of the following scenarios:

    - If we only consider the latest quote received by each vendor for each program, what is the total server rack cost per program per vendor?
    - If we only consider the first quote received by each vendor for each program, what is the total cost per program per vendor?
    - To determine "best-in-class" pricing, calculate the total server rack cost by determining the lowest price per component and summing the total, regardless of vendor. What is the "best-in-class" pricing for each program?

In [39]:
import polars as pl

# import pandas as pd
quote_lines_raw = pl.read_csv("quote_lines.csv")
quote_summaries_raw = pl.read_csv("quote_summaries.csv")
server_specs_raw = pl.read_excel('/Users/sarahkilpatrick/Documents/DATA 516 Homework 1/program configurations.xlsx', sheet_name='server_specs')
rack_specs_raw = pl.read_excel('/Users/sarahkilpatrick/Documents/DATA 516 Homework 1/program configurations.xlsx', sheet_name='rack_specs')

def clean_program_column(df: pl.DataFrame) -> pl.DataFrame:
    # Remove the "Program_" prefix from the Program column
    df_cleaned = df.with_columns(
        pl.col("Program").str.replace("Program_", "")  # Removing the "Program_" prefix
    )

    return df_cleaned

quote_lines_raw = clean_program_column(quote_lines_raw)

print("Raw quote_lines\n", quote_lines_raw.head())
print("Raw quote_summaries\n",quote_summaries_raw.head())
print("Raw server_specs\n",server_specs_raw)
print("Raw rack_specs\n",rack_specs_raw)


quote_summaries_raw = quote_summaries_raw.with_columns(
    pl.col("quote_timestamp").str.to_datetime("%Y-%m-%dT%H:%M:%S%.f")
)

# Server factors for each program
servers_factors = {'A': 12, 'B': 8, 'C': 14, 'D': 10, 'E': 10}

# Multiply each column by its corresponding factor
scaled_factors = server_specs_raw.select(
    [pl.col("Item")] +  # Keep the "Item" column as is
    [(pl.col(f"Server {server}") * factor).alias(f"Server {server}")
     for server, factor in servers_factors.items()]
)

# Display the result
print("Scaled factors", scaled_factors)

#Concatenate the scaling factors together from server specs and rack specs
rack_specs_filtered = rack_specs_raw.filter(pl.col('Item').is_in(['TOR', 'CHASSIS']))

scaling_factors = pl.concat([server_specs_raw, rack_specs_filtered], how='vertical')

scaling_factors_unpivoted = scaling_factors.unpivot(index="Item", variable_name="Program", value_name="Count")
def clean_program_column(df: pl.DataFrame) -> pl.DataFrame:
    # Remove the "Program_" prefix from the Program column
    df_cleaned = df.with_columns(
        pl.col("Program").str.replace("Server ", "")  # Removing the "Program_" prefix
    )

    return df_cleaned
scaling_factors_unpivoted = clean_program_column(scaling_factors_unpivoted)
print("scaling factors unpivoted\n", scaling_factors_unpivoted)

quote_lines_unpivoted = quote_lines_raw.unpivot(index=["Program", "Vendor", "quote_timestamp"], variable_name="Item", value_name="Price")
print("quote lines unpivoted\n", quote_lines_unpivoted)


Raw quote_lines
 shape: (5, 14)
┌──────────┬─────────┬────────────────────────────┬────────┬───┬────────┬───────┬────────┬─────────┐
│ Vendor   ┆ Program ┆ quote_timestamp            ┆ CPU    ┆ … ┆ PSU    ┆ TRAY  ┆ TOR    ┆ CHASSIS │
│ ---      ┆ ---     ┆ ---                        ┆ ---    ┆   ┆ ---    ┆ ---   ┆ ---    ┆ ---     │
│ str      ┆ str     ┆ str                        ┆ f64    ┆   ┆ f64    ┆ f64   ┆ f64    ┆ f64     │
╞══════════╪═════════╪════════════════════════════╪════════╪═══╪════════╪═══════╪════════╪═════════╡
│ Vendor_7 ┆ D       ┆ 2024-09-25T03:52:19.637095 ┆ 305.84 ┆ … ┆ 94.19  ┆ 11.57 ┆ 746.67 ┆ 1015.84 │
│ Vendor_6 ┆ E       ┆ 2024-09-12T00:26:22.414219 ┆ 324.21 ┆ … ┆ 89.99  ┆ 70.05 ┆ 728.08 ┆ 1068.57 │
│ Vendor_6 ┆ A       ┆ 2024-09-01T04:29:41.575326 ┆ 325.75 ┆ … ┆ 106.22 ┆ 28.0  ┆ 719.14 ┆ 1665.41 │
│ Vendor_5 ┆ A       ┆ 2024-09-23T18:35:08.078890 ┆ 300.29 ┆ … ┆ 102.29 ┆ 22.98 ┆ 734.33 ┆ 1770.98 │
│ Vendor_5 ┆ B       ┆ 2024-09-24T11:33:41.034306 ┆ 333.53 

# To-Do:
- see why the datetime filter gets rid of 8 records
- see why the vendor 7 filter gets rid of 12 records
- see if the scaled_total_cost matches with reported_total_cost from quote_summaries

# Clean Data 

There is little to no validation performed on quotes when they enter our system. As such, we need to perform the following validations on our quote data:

- Quotes for a given month can only be accepted starting on the first Monday of the month and ending on the 25th. 

    - These boundary dates are inclusive. 
        - Any quotes provided outside of these dates will not be considered.

    - The sum of the quote line details must match the provided quote summary total. 
        - If the two datasets do not agree, the quotes will not be considered.

- We do not purchase Program C and Program E quotes from Vendor 7. 

    - Vendor 7’s systems are configured to quote all available programs, so these quotes need to be discarded.


In [30]:
from datetime import datetime
import polars as pl

# Convert the quote_timestamp to datetime and filter the range
quote_lines_dated = (
    quote_lines_raw
    .with_columns(
        pl.col("quote_timestamp").str.to_datetime("%Y-%m-%dT%H:%M:%S%.f")
    )
    .filter(
        pl.col("quote_timestamp")
        .is_between(datetime(2024, 9, 2), datetime(2024, 9, 26))
    )

)
# Filter rows outside the above date range
discarded_rows = (
    quote_lines_raw
    .with_columns(
        pl.col("quote_timestamp").str.to_datetime("%Y-%m-%dT%H:%M:%S%.f")
    )
    .filter(
        ~pl.col("quote_timestamp").is_between(datetime(2024, 9, 2), datetime(2024, 9, 26))
    )
)

print(quote_lines_dated)

if discarded_rows.height > 0:
    print(f"Discarding {discarded_rows.height} quotes outside the date range:\n")
    print(discarded_rows.select(["Vendor", "Program", "quote_timestamp"]))



shape: (167, 14)
┌──────────┬─────────┬────────────────────────────┬────────┬───┬────────┬───────┬────────┬─────────┐
│ Vendor   ┆ Program ┆ quote_timestamp            ┆ CPU    ┆ … ┆ PSU    ┆ TRAY  ┆ TOR    ┆ CHASSIS │
│ ---      ┆ ---     ┆ ---                        ┆ ---    ┆   ┆ ---    ┆ ---   ┆ ---    ┆ ---     │
│ str      ┆ str     ┆ datetime[μs]               ┆ f64    ┆   ┆ f64    ┆ f64   ┆ f64    ┆ f64     │
╞══════════╪═════════╪════════════════════════════╪════════╪═══╪════════╪═══════╪════════╪═════════╡
│ Vendor_7 ┆ D       ┆ 2024-09-25 03:52:19.637095 ┆ 305.84 ┆ … ┆ 94.19  ┆ 11.57 ┆ 746.67 ┆ 1015.84 │
│ Vendor_6 ┆ E       ┆ 2024-09-12 00:26:22.414219 ┆ 324.21 ┆ … ┆ 89.99  ┆ 70.05 ┆ 728.08 ┆ 1068.57 │
│ Vendor_5 ┆ A       ┆ 2024-09-23 18:35:08.078890 ┆ 300.29 ┆ … ┆ 102.29 ┆ 22.98 ┆ 734.33 ┆ 1770.98 │
│ Vendor_5 ┆ B       ┆ 2024-09-24 11:33:41.034306 ┆ 333.53 ┆ … ┆ 113.9  ┆ 80.13 ┆ 616.13 ┆ 1563.11 │
│ Vendor_7 ┆ C       ┆ 2024-09-15 13:20:11.557807 ┆ 324.79 ┆ … ┆ 86.8   ┆ 

In [31]:
import polars as pl

def remove_vendor_7_programs(df):
    # Filter out rows where Vendor is 'Vendor_7' and Program is either 'Program_C' or 'Program_E'
    df_filtered = df.filter(
        ~(
            (pl.col("Vendor") == "Vendor_7") &
            (pl.col("Program").is_in(["C", "E"]))
        )
    )

    discarded_rows = df.filter(
        (pl.col("Vendor") == "Vendor_7") &
        (pl.col("Program").is_in(["C", "E"]))
    )

    if discarded_rows.height > 0:
        print(f"Discarding {discarded_rows.height} quotes from Vendor_7 for Program C and Program E:\n")
        print(discarded_rows.select(["Vendor", "Program", "quote_timestamp"]))

    return df_filtered

# Remove Vendor_7 from Program C or E
quote_lines_cleaned = remove_vendor_7_programs(quote_lines_dated)

print("Cleaned quote_lines\n", quote_lines_cleaned)

Discarding 12 quotes from Vendor_7 for Program C and Program E:

shape: (12, 3)
┌──────────┬─────────┬────────────────────────────┐
│ Vendor   ┆ Program ┆ quote_timestamp            │
│ ---      ┆ ---     ┆ ---                        │
│ str      ┆ str     ┆ datetime[μs]               │
╞══════════╪═════════╪════════════════════════════╡
│ Vendor_7 ┆ C       ┆ 2024-09-15 13:20:11.557807 │
│ Vendor_7 ┆ E       ┆ 2024-09-07 17:45:04.688250 │
│ Vendor_7 ┆ C       ┆ 2024-09-10 10:24:30.472069 │
│ Vendor_7 ┆ E       ┆ 2024-09-20 06:35:16.738483 │
│ Vendor_7 ┆ C       ┆ 2024-09-24 22:09:35.544275 │
│ …        ┆ …       ┆ …                          │
│ Vendor_7 ┆ C       ┆ 2024-09-04 20:54:28.430814 │
│ Vendor_7 ┆ E       ┆ 2024-09-13 12:18:38.436683 │
│ Vendor_7 ┆ C       ┆ 2024-09-05 08:48:50.223750 │
│ Vendor_7 ┆ E       ┆ 2024-09-02 19:59:44.793380 │
│ Vendor_7 ┆ C       ┆ 2024-09-10 20:59:39.959689 │
└──────────┴─────────┴────────────────────────────┘
Cleaned quote_lines
 shape: (155, 14

IndexError: list index out of range

Initial scaling factors DataFrame:
shape: (5, 12)
┌──────────┬─────┬─────┬─────┬───┬─────┬──────┬─────┬─────────┐
│ Program  ┆ CPU ┆ GPU ┆ RAM ┆ … ┆ PSU ┆ TRAY ┆ TOR ┆ CHASSIS │
│ ---      ┆ --- ┆ --- ┆ --- ┆   ┆ --- ┆ ---  ┆ --- ┆ ---     │
│ str      ┆ i64 ┆ i64 ┆ i64 ┆   ┆ i64 ┆ i64  ┆ i64 ┆ i64     │
╞══════════╪═════╪═════╪═════╪═══╪═════╪══════╪═════╪═════════╡
│ Server A ┆ 24  ┆ 0   ┆ 48  ┆ … ┆ 12  ┆ 12   ┆ 1   ┆ 1       │
│ Server B ┆ 16  ┆ 32  ┆ 32  ┆ … ┆ 16  ┆ 8    ┆ 2   ┆ 1       │
│ Server C ┆ 28  ┆ 0   ┆ 56  ┆ … ┆ 14  ┆ 14   ┆ 1   ┆ 1       │
│ Server D ┆ 10  ┆ 0   ┆ 80  ┆ … ┆ 10  ┆ 10   ┆ 1   ┆ 1       │
│ Server E ┆ 10  ┆ 20  ┆ 80  ┆ … ┆ 10  ┆ 10   ┆ 1   ┆ 1       │
└──────────┴─────┴─────┴─────┴───┴─────┴──────┴─────┴─────────┘
Columns before drop: ['Program', 'CPU', 'GPU', 'RAM', 'SSD', 'HDD', 'MOBO', 'NIC', 'PSU', 'TRAY', 'TOR', 'CHASSIS']
Columns after drop: ['Program', 'CPU', 'GPU', 'RAM', 'SSD', 'HDD', 'MOBO', 'NIC', 'PSU', 'TRAY', 'TOR', 'CHASSIS']
Final scaled Da

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/functools.py:877: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


In [23]:
import polars as pl
print(quote_summaries_raw.head())
quote_summaries_raw = quote_summaries_raw.with_columns(
    pl.col("quote_timestamp").str.to_datetime("%Y-%m-%dT%H:%M:%S%.f")
)

# Check the updated DataFrames
print("Updated quote_lines_scaled:\n", quote_lines_scaled.head())
print("Updated quote_summaries_raw:\n", quote_summaries_raw.head())

# Perform the join
joined_df = quote_lines_scaled.join(
    quote_summaries_raw,
    on=["vendor", "program", "quote_timestamp"],
    how="inner"  # Change this to "left", "right", or "outer" if needed
)

# Display the result of the join
print("Joined DataFrame:\n", joined_df)


shape: (5, 4)
┌──────────┬─────────┬────────────────────────────┬──────────────────────┐
│ vendor   ┆ program ┆ quote_timestamp            ┆ reported_total_price │
│ ---      ┆ ---     ┆ ---                        ┆ ---                  │
│ str      ┆ str     ┆ datetime[μs]               ┆ f64                  │
╞══════════╪═════════╪════════════════════════════╪══════════════════════╡
│ Vendor_3 ┆ B       ┆ 2024-09-15 14:42:25.510821 ┆ 29242.12             │
│ Vendor_5 ┆ E       ┆ 2024-09-23 04:24:43.218127 ┆ 40514.23             │
│ Vendor_5 ┆ E       ┆ 2024-09-03 20:14:36.055518 ┆ 57892.1              │
│ Vendor_1 ┆ D       ┆ 2024-09-24 20:49:52.296317 ┆ 50409.12             │
│ Vendor_4 ┆ C       ┆ 2024-09-08 16:45:01.071635 ┆ 19358.08             │
└──────────┴─────────┴────────────────────────────┴──────────────────────┘
Updated quote_lines_scaled:
 shape: (5, 26)
┌──────────┬─────────┬────────────┬────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ vendor   ┆ pro

In [24]:
# Filter rows where Scaled_Component_Sum is equal to reported_total_price
filtered_df = joined_df.filter(pl.col("Scaled_Component_Sum") == pl.col("reported_total_price"))

# Display the filtered DataFrame
print("Filtered DataFrame:\n", filtered_df)


Filtered DataFrame:
 shape: (0, 27)
┌────────┬─────────┬──────────────┬─────┬───┬────────────┬─────────────┬─────────────┬─────────────┐
│ vendor ┆ program ┆ quote_timest ┆ CPU ┆ … ┆ TOR_scaled ┆ CHASSIS_sca ┆ Scaled_Comp ┆ reported_to │
│ ---    ┆ ---     ┆ amp          ┆ --- ┆   ┆ ---        ┆ led         ┆ onent_Sum   ┆ tal_price   │
│ str    ┆ str     ┆ ---          ┆ f64 ┆   ┆ f64        ┆ ---         ┆ ---         ┆ ---         │
│        ┆         ┆ datetime[μs] ┆     ┆   ┆            ┆ f64         ┆ f64         ┆ f64         │
╞════════╪═════════╪══════════════╪═════╪═══╪════════════╪═════════════╪═════════════╪═════════════╡
└────────┴─────────┴──────────────┴─────┴───┴────────────┴─────────────┴─────────────┴─────────────┘
